In [1]:
import pandas as pd
import numpy as np
from dateutil import relativedelta
pd.options.display.max_rows = 100
from datetime import datetime
import re
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import scipy
import json

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn import metrics

from sklearn import ensemble
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
import warnings
warnings.filterwarnings('ignore')

## local imports

from os import pardir, path
import sys
mod_path = path.abspath(path.join(pardir))
print(mod_path)
if mod_path not in sys.path:
    sys.path.append(mod_path)
print(sys.path)

from data_extract import client, fin, company , companyfinancials_list, pnl, bs, cf, profile, prices, fx, mrktcap, prices_n_fin,  mldf, mlstat, MONGO_FIN ## note needed for this notebook ignore if not using logal Mongodb
from mypackage import Feature, Model, calperiod

/Users/karimkhalil/Coding/development/fin
['/Users/karimkhalil/Coding/development/fin/notebooks', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python39.zip', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python3.9', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python3.9/lib-dynload', '', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python3.9/site-packages', '/Users/karimkhalil/Coding/development/fin']
['admin', 'config', 'db_fin', 'local']
['cf', 'prices_bckup', 'company_today', 'mrktcap', 'company_wfinancials', 'prices_n_fin', 'bs', 'fx', 'profile', 'idx_price', 'prices', 'idx_all', 'pnl']
mypackage imported


In [3]:
df_final_vf = pd.read_csv('../data/df_final_vf_old.csv')
model = Model(df_final_vf)
l_train = model.train(model.df,'cal_yearperiod', 8,1,0,True)

regr = LinearRegression()
ridge_regr = Ridge()
xgb = ensemble.GradientBoostingRegressor()
histgrad = ensemble.HistGradientBoostingRegressor()
randforest = ensemble.RandomForestRegressor()

cols_y =  'usd_adjClose_t-1_change'
cols_x = ['CASH2ASSETS_ratio_t-1_change_indavg', 'CF2CL_ratio_t-1_change_excess_indavg', 'ROA_EBITDA_ratio_indavg_t-1_change_indavg', 'usd_adjClose_t-2_change_excess_indavg' , 'usd_adjClose_t-2_change_excess_globalavg']

model object instantiated
loaded DataFrame with size 207874 rows : 224 columns


In [ ]:
data_train_regr , data_test_regr, stat_train_regr, stat_test_regr = model.skpredict_window(regr, cols_x, cols_y, col_period='cal_yearperiod' , train_window=8 , test_window=1, test_gap = 0, expanding=True)

In [ ]:
rsquare_reg = [i['r2'] for i in data_train_regr]
print(np.mean(rsquare_reg))

In [ ]:
data_train_regr['model'] = "data_train_regr"
data_test_regr['model'] = "data_test_regr"

for i in stat_train_regr:
    i.update({"model" : "data_train_regr"})

for i in stat_test_regr:
    i.update({"model" : "data_test_regr"})

In [ ]:
data_train_histgrand , data_test_histgrad, stat_train_histgrand, stat_test_histgrand = model.skpredict_window(histgrad, cols_x, cols_y, col_period='cal_yearperiod' , train_window=8 , test_window=1, test_gap = 0, expanding=True)

In [ ]:
data_train_histgrand['model'] = "data_train_histgrand"
data_test_histgrad['model'] = "data_test_histgrad"

for i in stat_train_histgrand:
    i.update({"model" : "data_train_histgrand"})

for i in stat_test_histgrand:
    i.update({"model" : "data_test_histgrad"})

In [4]:
data_train_xgb , data_test_xgb, stat_train_xgb, stat_test_xgb = model.skpredict_window(xgb, cols_x, cols_y, col_period='cal_yearperiod' , train_window=8 , test_window=1, test_gap = 0, expanding=True)

loaded DataFrame with size 207874 rows : 224 columns
train stat: {'mse': 0.016403345922166943, 'mae': 0.09701808775544377, 'r2': 0.813696808405701, 'window': 0, 'date': '1991Q1-1991Q4'}
test stat: {'mse': 0.23243389857642563, 'mae': 0.20602032232387468, 'window': 0, 'date': '1992Q1-1992Q1'}

train stat: {'mse': 0.023579563731420443, 'mae': 0.11235625966397378, 'r2': 0.8255582822892426, 'window': 1, 'date': '1991Q1-1991Q4'}
test stat: {'mse': 0.04218430033048798, 'mae': 0.14063771421342092, 'window': 1, 'date': '1992Q2-1992Q2'}

train stat: {'mse': 0.02502445015810571, 'mae': 0.11335198350039004, 'r2': 0.7909684830253034, 'window': 2, 'date': '1991Q1-1991Q4'}
test stat: {'mse': 0.028710589572178053, 'mae': 0.12711892743009304, 'window': 2, 'date': '1992Q3-1992Q3'}

train stat: {'mse': 0.02405271425869902, 'mae': 0.11358980918077406, 'r2': 0.7808367714822286, 'window': 3, 'date': '1991Q1-1991Q4'}
test stat: {'mse': 0.054741189464856496, 'mae': 0.1405867223092808, 'window': 3, 'date': '19

In [15]:
stat_train_xgb['iteration']= stat_train_xgb['iteration'].astype(int)

TypeError: list indices must be integers or slices, not str

In [5]:
rsquare_xgb = [i['r2'] for i in stat_train_xgb]
print(np.mean(rsquare_xgb))

0.8954832078881334


In [11]:
type(data_train_xgb.iteration.unique()[-1])

numpy.int64

In [18]:
data_test_xgb[data_test_xgb['iteration'] == 120]['cal_yearperiod'].unique()

array(['2022Q1'], dtype=object)

In [63]:
data_port = data_train_xgb.loc[(data_train_xgb['year'] >= 2018) & (data_test_xgb['iteration'] == 120)]
data_port

: 

: 

In [52]:
data_stocks = data_port.groupby('symbol')['usd_adjClose_t-1_change','predict', 'MSE'].mean().reset_index()

In [53]:
data_stocks_mse = data_stocks.nsmallest(n=100, columns=['MSE'])
data_stocks_mse.head(5)

,symbol,usd_adjClose_t-1_change,predict,MSE


In [54]:
data_stocks_returns = data_stocks_mse.nlargest(n=20, columns = ['usd_adjClose_t-1_change'])
data_stocks_returns

,symbol,usd_adjClose_t-1_change,predict,MSE


In [39]:
stocks = data_stocks_returns.symbol.unique().tolist()

In [45]:
print(stocks)

['1185.HK', 'PGHH.NS', 'PIDILITIND.NS', 'APGN.SW', 'ESUN.SW', '7239.T', 'MERCATOR.NS', '8877.T', 'TASB.ME', 'SAIC', 'SSWL.NS', 'LACR.PA', '4708.T', 'ICT.AS', '0270.HK', '6877.T', 'VKTX', '603669.SS', '002716.SZ', 'AUPH']


In [42]:
df_full = pd.read_csv('../data/data_vf.csv')
df_full.shape

(608641, 128)

In [43]:
df_port = df_full.loc[df_full['symbol'].isin(stocks)]

In [50]:
df_port2 = df_port.groupby(['symbol', 'cal_yearperiod'])['usd_adjClose'].count().reset_index()
df_port2

,symbol,cal_yearperiod,usd_adjClose
0,002716.SZ,2014Q1,1
1,002716.SZ,2014Q2,1
2,002716.SZ,2014Q3,1
3,002716.SZ,2014Q4,1
4,002716.SZ,2015Q1,1
...,...,...,...
309,VKTX,2021Q1,1
310,VKTX,2021Q2,1
311,VKTX,2021Q3,1
312,VKTX,2021Q4,1


In [49]:
df_port3 = df_port2.groupby(['cal_yearperiod'])['symbol'].count().reset_index()
df_port3

,cal_yearperiod,symbol
0,2008Q4,2
1,2009Q1,4
2,2009Q3,4
3,2009Q4,4
4,2010Q1,5
5,2010Q2,9
6,2010Q3,5
7,2010Q4,4
8,2011Q1,5
9,2011Q2,7
